# Notebook Overview
This notebook serves as an archive for some of the initial testing and experimentation done on the dataset that is not described but very much had an impact on the methods in the written report. This code is messy and is **not** expected to work; it is provided as a proof of concept what remains from the initial investigation.

Below is an example of an initial model architecture. Note that this was before the data was cleaned/when the baseline class was being separated from left and right.

In [ ]:
from tensorflow.keras.layers import Reshape, LSTM

def build_cnn_lstm(input_shape, num_classes=10):
  model = Sequential()

  model.add(Conv2D(16, (5, 3), activation='relu', padding='same', input_shape=input_shape))
  model.add(Conv2D(16, (5, 3), activation='relu', padding='same', input_shape=input_shape))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.1))

  model.add(Conv2D(32, (7, 3), activation='relu', padding='same'))
  model.add(Conv2D(32, (7, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.1))

  # Flattening CNN output
  model.add(Flatten())

  # Reshaping for LSTM (time steps, features)
  model.add(Reshape((-1, 32)))  # Adjust the second dimension based on your CNN output size

  # LSTM Layers (Temporal Sequence Learning)
  model.add(LSTM(32, return_sequences=True))
  model.add(LSTM(32))
  model.add(Dropout(0.2))

  # Dense Layers for Classification
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(9, activation='softmax'))  # Assuming 10 classes for classification

  return model

top_channels_model = build_cnn_lstm((101, 4, top_k), num_classes=10)
top_channels_model.summary()

Belwo are notes during a period where I was experimenting with different kernel sizes: These are the top accuracies achieved at different combinations. Note that this was also before some vital accuracy increasing steps, such as data cleaning, final decisions on classes, etc. Some results are slightly confusing, as kernel sizes expand far beyond the time dimension, but it does suggest that kernel much larger than those used can be considered in a more complete grid search.


*In general validation loss is staying quite high*

*full 11 x 11 kernels: 41.5%*

*11 x 11 -> 15 x 15: 54.6%*

*13 x 13 -> 19 x 19: 40%*

*13, 15 -> 19, 23: 33.7%*

*11-> 31: 59.4%*

*11 x 3 -> 31 x 5: 58.7%*

*11 x 5 -> 31 x 7: 65.1%*

*11 x 5 -> 51 x 7: 65.4%*

*11 x 5 -> 51 x 11: 66.3%*

*11 x 5 -> 51 x 9: 55.9%*

*31 x 5 -> 51 x 11: 60.3%*

*11 x 5, 31 x 9, 51 x 15: 50.8%*

*11 x 5, 31 x 9, 51 x 11: 55. 2%*

*41 x 5, 51 x 7: 46%*

*3 x 3 -> 11 x 5 -> 51 x 11: 55.6%*

*5 x 5 -> 11 x 5 -> 51 x 11: 61.6%*

*1 x 1 -> 11 x 5 -> 51 x 11: 64.8%*

*11 x 3, 3 x 2 -> 64.8%*

Below is a sample from initial training on PCA. In this case, training actually did not occur at all, leading to an initial hypothesis that the encoding method was by far the superior. Once our methods were finalized this proved to not be the case.

In [ ]:
# Training autoencoder + CNN

optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
batch_size = 128
epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15,
                                                  restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.95,
                              patience=5, min_lr=0.00001)

pca_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

hist = pca_model.fit(X_pca_train, y_pca_train, callbacks=[early_stopping, reduce_lr],
                 epochs=epochs, batch_size=batch_size, validation_split=0.2)


Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 718ms/step - accuracy: 0.0955 - loss: 1068.4323 - val_accuracy: 0.0794 - val_loss: 57.5287 - learning_rate: 0.0010
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.1117 - loss: 29.4635 - val_accuracy: 0.1270 - val_loss: 2.2998 - learning_rate: 0.0010
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.0868 - loss: 2.4491 - val_accuracy: 0.0992 - val_loss: 2.3026 - learning_rate: 0.0010
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0872 - loss: 2.3063 - val_accuracy: 0.0992 - val_loss: 2.3026 - learning_rate: 0.0010
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.1046 - loss: 2.3258 - val_accuracy: 0.0992 - val_loss: 2.3026 - learning_rate: 0.0010
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0944 - loss: 2.3026 - val_accuracy: 0.0992 - val_loss: 2.3026 - learning_rate: 0.0010
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.1052 - loss: 2.3024 - val_accuracy: 0.0992 - val_loss: 2.3026 - learning_rate: 0.0010
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0956 - loss: 2.3024 - val_accuracy: 0.1032 - val_loss: 2.3026 - learning_rate: 9.5000e-04
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.1070 - loss: 2.3672 - val_accuracy: 0.1032 - val_loss: 2.3025 - learning_rate: 9.5000e-04
Epoch 10/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.1107 - loss: 2.4188 - val_accuracy: 0.1032 - val_loss: 2.3025 - learning_rate: 9.5000e-04
Epoch 11/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.0963 - loss: 2.3027 - val_accuracy: 0.1032 - val_loss: 2.3025 - learning_rate: 9.5000e-04
Epoch 12/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0977 - loss: 2.3025 - val_accuracy: 0.1032 - val_loss: 2.3025 - learning_rate: 9.5000e-04
Epoch 13/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.1039 - loss: 2.3110 - val_accuracy: 0.1032 - val_loss: 2.3026 - learning_rate: 9.0250e-04
Epoch 14/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.1091 - loss: 2.3022 - val_accuracy: 0.1032 - val_loss: 2.3026 - learning_rate: 9.0250e-04
Epoch 15/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.1026 - loss: 2.3122 - val_accuracy: 0.1032 - val_loss: 2.3026 - learning_rate: 9.0250e-04
Epoch 16/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.0997 - loss: 2.3033 - val_accuracy: 0.1032 - val_loss: 2.3026 - learning_rate: 9.0250e-04
Epoch 17/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.1186 - loss: 2.3020 - val_accuracy: 0.1032 - val_loss: 2.3026 - learning_rate: 9.0250e-04

Below is the code that was used to intitially test a full autoencoder trained without any link to a classifier. This model (and the classifier then tried on the encoding data) performed much worse than the singular model.

In [ ]:
epoched_total = epoched_total.transpose(2, 3, 1, 0)
epoched_total = epoched_total.reshape(-1, 508, 256)

import tensorflow as tf
from tensorflow.keras import layers, models

input_shape = (51, 5, 256)

inputs = layers.Input(shape=input_shape)

# --- ENCODER ---
x = layers.Conv2D(128, (5, 1), activation='relu', padding='same', name='encoded_1')(inputs)
x = layers.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
bottleneck = layers.Conv2D(32, (3, 1), activation='relu', padding='same', name='bottle')(x)

# --- DECODER ---
x = layers.Conv2D(64, (5, 1), activation='relu', padding='same')(bottleneck)
x = layers.Conv2D(128, (5, 1), activation='relu', padding='same')(x)
decoded = layers.Conv2D(256, (5, 1), activation='linear', padding='same')(x)
# Model
autoencoder = models.Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()


X_auto = specs.copy()
print(X_auto.shape)
#X_auto = (X_auto - np.mean(X_auto)) / np.std(X_auto)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15,
                                                  restore_best_weights=True)

autoencoder.fit(X_auto, X_auto,  # X is both input and target
                epochs=100,
                batch_size=16,
                validation_split=0.1,
                callbacks=[early_stopping],
                shuffle=True)

# Extract encoder model
# Assume autoencoder is your trained model
encoder = models.Model(autoencoder.input, autoencoder.get_layer('bottle').output)

# Apply the encoder to your data (e.g., X_train)
decoded_data = encoder.predict(specs)
#decoded_data = decoded_data.transpose(0, 2, 1)

decoded_data.shape

Below are early examples of testing model architectures.

In [2]:
# 2D CNN for training on spectrograms
input_shape = (51, 5, 32)
output_shape = 5

cnn = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(512, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.Conv2D(512, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.1),
        layers.Conv2D(1024, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.Conv2D(1024, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
        layers.Dropout(0.1),
        layers.Conv2D(2048, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.Conv2D(2048, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
        layers.Dropout(0.1),
        layers.Flatten(),
        layers.Dense(1024, activation="relu"),
        layers.Dropout(0.1),
        layers.Dense(output_shape, activation="softmax"),
    ]
)

cnn.summary()

# LSTM
input_shape = (13, 33, 32)  # (time, freq, channels)
output_shape = 5

lstm = keras.Sequential([
    # Input shape: (51 time steps, 5 freq bins, 256 channels)
    layers.Input(shape=input_shape),

    # 1. Frequency-domain processing (applied per time step)
    layers.TimeDistributed(layers.Conv1D(32, 3, activation='relu', padding='same')),
    layers.TimeDistributed(layers.Conv1D(32, 3, activation='relu', padding='same')),
    layers.TimeDistributed(layers.MaxPooling1D(2)),
    #layers.TimeDistributed(layers.Dropout(0.1)),

    # 2. Deeper frequency processing
    layers.TimeDistributed(layers.Conv1D(64, 3, activation='relu', padding='same')),
    layers.TimeDistributed(layers.Conv1D(64, 3, activation='relu', padding='same')),
    layers.TimeDistributed(layers.MaxPooling1D(2)),
    layers.TimeDistributed(layers.Dropout(0.1)),

    layers.TimeDistributed(layers.Conv1D(128, 3, activation='relu', padding='same')),
    layers.TimeDistributed(layers.Conv1D(128, 3, activation='relu', padding='same')),
    layers.TimeDistributed(layers.MaxPooling1D(2, padding='same')),
    layers.TimeDistributed(layers.Dropout(0.15)),

    # 3. Prepare for LSTM - output will be (None, 51, remaining_features)
    layers.TimeDistributed(layers.Flatten()),

    # 4. Temporal processing
    layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(128)),
    layers.Dropout(0.4),

    # 5. Classifier
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(output_shape, activation='softmax')
])

lstm.summary()

'# 2D CNN for training on spectrograms\ninput_shape = (51, 5, 32)\noutput_shape = 5\n\ncnn = keras.Sequential(\n    [\n        keras.Input(shape=input_shape),\n        layers.Conv2D(512, kernel_size=(3, 3), activation="relu", padding="same"),\n        layers.Conv2D(512, kernel_size=(3, 3), activation="relu", padding="same"),\n        layers.MaxPooling2D(pool_size=(2, 2)),\n        layers.Dropout(0.1),\n        layers.Conv2D(1024, kernel_size=(3, 3), activation="relu", padding="same"),\n        layers.Conv2D(1024, kernel_size=(3, 3), activation="relu", padding="same"),\n        layers.MaxPooling2D(pool_size=(2, 2), padding=\'same\'),\n        layers.Dropout(0.1),\n        layers.Conv2D(2048, kernel_size=(3, 3), activation="relu", padding="same"),\n        layers.Conv2D(2048, kernel_size=(3, 3), activation="relu", padding="same"),\n        layers.MaxPooling2D(pool_size=(2, 2), padding=\'same\'),\n        layers.Dropout(0.1),\n        layers.Flatten(),\n        layers.Dense(1024, activati

Below is some leftover code from when we were trying to create our spectrograms dynamically within our full model (because the initial idea was to perform encoding on the raw data). Eventually this was decided to be a step that was not only unneccesarily complicated but perhaps unnecessary overall.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

# class to support creating spectrograms within tf
class EEGSpectrogramLayer(Layer):
    def __init__(self, fs=500, nperseg=100, noverlap=0, **kwargs):
        super(EEGSpectrogramLayer, self).__init__(**kwargs)
        self.fs = fs
        self.nperseg = nperseg
        self.noverlap = noverlap

    def call(self, inputs):
        def _compute_spec(x_np):
            specs = []
            for i in range(x_np.shape[0]):  # loop over neurons
                f, t, Sxx = scipy.signal.spectrogram(
                    x_np[i],
                    fs=self.fs,
                    nperseg=self.nperseg,
                    noverlap=self.noverlap,
                )
                specs.append(Sxx)
            return np.stack(specs, axis=-1)  # Shape: (freq, time, neurons)

        specs = tf.numpy_function(_compute_spec, [inputs], tf.float32)
        specs.set_shape([None, None, inputs.shape[0]])  # freq, time, channels
        return specs


Below is the very first training performed on the raw data using five classes. This archive is particularly useful in that it demonstrates how much relatively easier it is to achieve high accuracy on only five classes, as well the early suggestions that our models could get away with relatively few parameters given the data complexity.

In [ ]:
# 1D CNN architecture that trains directly on data loaded using Mike's functions

input_shape = (508, total_trials)
output_shape = 5

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv1D(128, kernel_size=3, activation="relu", padding="same"),
        layers.Conv1D(128, kernel_size=5, activation="relu", padding="same"),
        layers.Conv1D(128, kernel_size=7, activation="relu", padding="same"),
        layers.Dropout(0.75),
        layers.MaxPooling1D(pool_size=3),
        layers.Conv1D(256, kernel_size=3, activation="relu", padding="same"),
        layers.Conv1D(256, kernel_size=5, activation="relu", padding="same"),
        layers.Dropout(0.75),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(512, kernel_size=3, activation="relu", padding="same"),
        layers.Dropout(0.75),
        layers.MaxPooling1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.Dropout(0.75),
        layers.Dense(output_shape, activation="softmax"),
    ]
)

model.summary()

### Best accuracies for differnet parameter numbers
##### Note: 5 and 10 million use 2D
best_5_million = 0.863

best_10_million = 0.867

best_12_million = 0.82

best_3_million = 0.852

best_1_1_million = 0.84

best_780K = 0.848

best_600K = 0.852

best_550K = 0.84

best_376K = 0.848

best_285K = 0.82
#### Overfitting seems to be less of a problem around here
best_200K = 0.84

best_98K = 0.796

best_76K = 0.793

best_54K = 0.801

best_37K = 0.77

best_26K = 0.84

best_15K =  0.781

best_12K = 0.801

best_4K = 0.738

This is a small note on early testing results. Important is its suggestion that LSTMs are useful in learning the data.

'''
Saving best spectorgram results here
separating labels for left and right top accuracy is 56.8%
10 labels and LSTM -best is 67.9%
generally got about 80% with different models using all data and 5 classes
'''

Below is an early and poor attempt at making spectrograms

In [2]:
#  New FFT functions
from scipy.signal import spectrogram
print(epoched_total.shape)
X_multichannel = epoched_total.reshape(1575, 508, 256, 1) #.reshape(130048, 175, 1) Ensure the shape matches (508, 35, 1)

X = epoched_total.transpose(2, 3, 0, 1).reshape(-1, 256, 508)  # (175, 256, 508)
y_multichannel = np.tile(np.arange(5), 315)  # Labels: [0, 1, 2, 3, 4, 0, 1, 2, ...]

print("New X shape:", X_multichannel.shape)  # (175, 256, 508)
print("New y shape:", y_multichannel.shape)  # (175,)

def compute_eeg_fft_spectrogram(eeg_data, fs=250, window_size=25, step_size=10, fmax=50):
  num_channels, num_samples = eeg_data.shape
  freqs = np.fft.rfftfreq(window_size, d=1/fs)
  max_bin = np.argmax(freqs > fmax) if np.any(freqs > fmax) else len(freqs)
  num_windows = (num_samples - window_size) // step_size + 1
  spec = np.zeros((num_channels, max_bin, num_windows))

  for ch in range(num_channels):
    for i in range(num_windows):
      start = i * step_size
      end = start + window_size
      segment = eeg_data[ch, start:end]
      fft_vals = np.abs(rfft(segment))[:max_bin]
      spec[ch, :, i] = fft_vals

  return spec, freqs[:max_bin]

fmax = 50

ffts_images = []
for i in range(X.shape[0]):# Get a longer EEG window (1 second)
  real_epoched = X[i, :, :, 0].T  # (channels, samples)

  fft_img = compute_eeg_fft_spectrogram(real_epoched, fs=250, window_size=25, step_size=10, fmax=50)[0]
  ffts_images.append(fft_img)

plt.figure(figsize=(10, 6))
plt.imshow( np.log1p(spectrogram[0]), origin='lower', aspect='auto',
    cmap='viridis', extent=[0, spectrogram.shape[2], 0, spectrogram.shape[1]])
plt.title(f"Spectrogram - Channel {ch_idx}")
plt.xlabel("Time Window")
plt.ylabel("Frequency (Hz)")
plt.colorbar(label="Log Magnitude")
plt.tight_layout()
plt.show()

This code was used to isolate the smoothing process and asses its impact on accuracy.

In [ ]:
# Code for smoothing
EEG_left_smoothed = np.empty_like(epoched_left)
for trial_idx in range(epoched_left.shape[2]):
    for class_idx in range(epoched_left.shape[3]):
        # Extract the time x channel data for the current trial and class
        eeg_trial_class = epoched_left[:, :, trial_idx, class_idx].T # Transpose to get (time, channels)
        # Apply smoothing and assign to the new array
        EEG_left_smoothed[:, :, trial_idx, class_idx] = smooth_outliers(eeg_trial_class).T # Transpose back to original shape

del epoched_left

EEG_right_smoothed = np.empty_like(epoched_right)
for trial_idx in range(epoched_right.shape[2]):
    for class_idx in range(epoched_right.shape[3]):
        # Extract the time x channel data for the current trial and class
        eeg_trial_class = epoched_right[:, :, trial_idx, class_idx].T # Transpose to get (time, channels)
        # Apply smoothing and assign to the new array
        EEG_right_smoothed[:, :, trial_idx, class_idx] = smooth_outliers(eeg_trial_class).T # Transpose back to original shape

del epoched_right